# 事前準備

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
# 読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 画像データ→行データ
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# 正規化
X_train = X_train.astype(float)
X_test = X_test.astype(float)
X_train /= 255
X_test /= 255

# one-hotベクトル化
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

# one-hotのデータ分割
X_train, X_val, y_train_one_hot, y_val_one_hot = train_test_split(X_train, y_train_one_hot, stratify=y_train_one_hot, test_size=0.2, random_state=0)

# 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造は前のSprintで作成した全結合層のFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。


ここでは パディング は考えず、ストライド も1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。


フォワードプロパゲーションの数式は以下のようになります。


$$a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b$$
$a_i$ : 出力される配列のi番目の値


$F$ : フィルタのサイズ


$x_{(i+s)}$ : 入力の配列の(i+s)番目の値


$w_s$ : 重みの配列のs番目の値


$b$ : バイアス項


すべてスカラーです。


次に更新式です。ここがAdaGradなどに置き換えられる点は全結合層と同様です。


$$w_s^{\prime} = w_s - \alpha \frac{\partial L}{\partial w_s} \\ b^{\prime} = b - \alpha \frac{\partial L}{\partial b}$$
$\alpha$ : 学習率


$\frac{\partial L}{\partial w_s}$ : $w_s$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial b}$ : $b$ に関する損失 $L$ の勾配


勾配 $\frac{\partial L}{\partial w_s}$ や $\frac{\partial L}{\partial b}$ を求めるためのバックプロパゲーションの数式が以下です。


$$\frac{\partial L}{\partial w_s} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}x_{(i+s)}\\ \frac{\partial L}{\partial b} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}$$
$\frac{\partial L}{\partial a_i}$ : 勾配の配列のi番目の値


$N_{out}$ : 出力のサイズ


前の層に流す誤差の数式は以下です。


$$\frac{\partial L}{\partial x_j} = \sum_{s=0}^{F-1} \frac{\partial L}{\partial a_{(j-s)}}w_s$$
$\frac{\partial L}{\partial x_j}$ : 前の層に流す誤差の配列のj番目の値


ただし、 $j-s<0$ または $j-s>N_{out}-1$ のとき $\frac{\partial L}{\partial a_{(j-s)}} =0$ です。


全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差をすべて足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。
## SimpleConv1dSimpleConv1d

# 【問題2】1次元畳み込み後の出力サイズの計算
畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。

$$N_{out} =  \frac{N_{in}+2P-F}{S} + 1$$

$N_{out}$ : 出力のサイズ（特徴量の数）


$N_{in}$ : 入力のサイズ（特徴量の数）


$P$ : ある方向へのパディングの数


$F$ : フィルタのサイズ


$S$ : ストライドのサイズ

In [3]:
def get_N_out(N_in, F_size, pading=0, storaid=1):
    """
    畳み込みを行った後の特徴量の数
    """
    
    # npではないのでpython組み込み関数を使用
    N_out = int((N_in + 2 * pading - F_size) / storaid + 1)
    
    return N_out

# 【問題3】小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。

In [4]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

In [5]:
# 出力サイズの確認
a = np.zeros(get_N_out(x.shape[0], w.shape[0]))
a

array([0., 0.])

In [6]:
# aの値を更新
for i in range(a.shape[0]):
    a[i] = int(x[i: i + w.shape[0]] @ w + b)
    
a.astype(int)

array([35, 50])

In [37]:
def forward(x, w, b):
    """順伝播
    Parameters
    -----------
    x : 入力配列
    w : 重み
    b : バイアス
    """
    # 返り値入力配列
    a = []
    # 1づつずらしながら畳み込み計算
    for i in range(len(w) - 1):
        a.append((x[i:i+len(w)] @ w) + b[0])

    return np.array(a)

In [39]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
forward(x, w, b)

array([35, 50])

In [7]:
delta_a = np.array([10, 20])
dW = np.zeros_like(w).astype(int)
dB = np.zeros_like(b).astype(int)

![Imgur](https://i.imgur.com/QoMNGU3l.jpg)

In [8]:
# 初期化
dW = np.zeros_like(w).astype(int)
dB = np.zeros_like(b).astype(int)

# 誤差の計算
for i in range(a.shape[0]):
    dW += delta_a[i] * x[i : i+w.shape[0]]
    dB += delta_a[i]

In [9]:
dW

array([ 50,  80, 110])

# backword

![Imgur](https://i.imgur.com/SICodEJl.jpg)

In [10]:
# 初期化
dX = np.zeros_like(x)

# 誤差の計算
for i in range(dX.shape[0]):
    for s in range(w.shape[0]):
        # 入力の枠外の微分は0とする
        if (i-s < 0) or (i-s > delta_a.shape[0]-1):
            pass
        # 枠内の微分値は足し合わせる
        else:
            dX[i] += delta_a[i-s] * w[s]

In [11]:
dX

array([ 30, 110, 170, 140])

In [49]:
# 逆伝播の値
dx = []
# 逆畳込み計算用配列
# 0を0番目(先頭)に配置
new_w = np.insert(w[::-1], 0, 0)
# 0を最後に配置
new_w = np.append(new_w, 0)

for i in range(len(new_w)-1):
    dx.append(new_w[i:i+len(delta_a)] @ delta_a)
dx = np.array(dx[::-1])
dx

array([ 30, 110, 170, 140])

In [63]:
# 逆転する
w[::-1]

array([7, 5, 3])

In [55]:
new_w

array([0, 7, 5, 3, 0])

In [117]:
def forward(X, W, b):
    """
    順伝播

    param
    --------------
    self.X : ndarray (n_features, )
    入力サイズ
    """

    # 形を作成(出力チャンネル、　出力ノード)
    A = np.zeros((3, 2))
    # Aの行を指定
    for i in range(A.shape[0]):
        # Aの列を指定
        for j in range(A.shape[1]):
            # Xのチャンネルを指定
            for k in range(X.shape[0]):
                # Xの形をWが存在するところまでに切り取る(Wにゼロを追加するのと同義)
                A[i, j] += X[k, j : j+w.shape[2]].T @ W[i, j]
            # バイアスの計算
            A[i, j] += b[i]

    A = A.astype(int)
    
    return A

In [167]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）

In [169]:
forward(x, w, b)

array([[16, 22],
       [17, 23],
       [18, 24]])

In [189]:
# 0埋め実施
x = np.pad(x, ((0,0), ((w.shape[2]-1), 0)))
x

array([[0, 0, 1, 2, 3, 4],
       [0, 0, 2, 3, 4, 5]])

In [201]:
X1 = np.zeros((2, w.shape[2], 4+(w.shape[2]-1)))
X1

array([[[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]])

In [191]:
A = np.zeros((3, 2))
A

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [203]:
# 重みの長さでループ
for i in range(w.shape[2]):
    # ずらしながら上書き
    X1[:, i] = np.roll(x, -i, axis=-1)

In [205]:
X1

array([[[0., 0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4., 0.],
        [1., 2., 3., 4., 0., 0.]],

       [[0., 0., 2., 3., 4., 5.],
        [0., 2., 3., 4., 5., 0.],
        [2., 3., 4., 5., 0., 0.]]])

In [215]:
X1.shape

(2, 3, 6)

In [223]:
X1

array([[[0., 0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4., 0.],
        [1., 2., 3., 4., 0., 0.]],

       [[0., 0., 2., 3., 4., 5.],
        [0., 2., 3., 4., 5., 0.],
        [2., 3., 4., 5., 0., 0.]]])

In [229]:
X1[:, :, w.shape[0]-1:4]

array([[[1., 2.],
        [2., 3.],
        [3., 4.]],

       [[2., 3.],
        [3., 4.],
        [4., 5.]]])

In [211]:
w[:, :, :, np.newaxis].shape

(3, 2, 3, 1)

In [227]:
w[:, :, :, np.newaxis]

array([[[[1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.]]],


       [[[1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.]]],


       [[[1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.]]]])

In [251]:
(X1[:, :, w.shape[0]-2:5]*w[:, :, :, np.newaxis])

array([[[[0., 1., 2., 3.],
         [1., 2., 3., 4.],
         [2., 3., 4., 0.]],

        [[0., 2., 3., 4.],
         [2., 3., 4., 5.],
         [3., 4., 5., 0.]]],


       [[[0., 1., 2., 3.],
         [1., 2., 3., 4.],
         [2., 3., 4., 0.]],

        [[0., 2., 3., 4.],
         [2., 3., 4., 5.],
         [3., 4., 5., 0.]]],


       [[[0., 1., 2., 3.],
         [1., 2., 3., 4.],
         [2., 3., 4., 0.]],

        [[0., 2., 3., 4.],
         [2., 3., 4., 5.],
         [3., 4., 5., 0.]]]])

In [247]:
np.sum(X1[:, :, w.shape[0]:5]*w[:, :, :, np.newaxis], axis=(1))

array([[[5., 7.],
        [7., 9.],
        [9., 0.]],

       [[5., 7.],
        [7., 9.],
        [9., 0.]],

       [[5., 7.],
        [7., 9.],
        [9., 0.]]])

In [239]:
np.sum(X1[:, :, w.shape[0]-1:4]*w[:, :, :, np.newaxis], axis=(2))

array([[[ 6.,  9.],
        [ 9., 12.]],

       [[ 6.,  9.],
        [ 9., 12.]],

       [[ 6.,  9.],
        [ 9., 12.]]])

# backword
![Imgur](https://i.imgur.com/EtJ2ZdWl.jpg)

In [75]:
# 初期化
dW = np.zeros_like(w).astype(int)
dB = np.zeros_like(b).astype(int)
X = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) 
dA = np.array([[16, 22], [17, 23], [18, 24]])

# 誤差の計算
for i in range(dW.shape[0]):
    for j in range(dW.shape[1]):
        for l in range(dA.shape[1]):
            dW[i, j] += dA[i, l] * X[j, l : l+dW.shape[2]]
            if j == 0:
                dB[i] += dA[i, l]

In [77]:
dW

array([[[ 60,  98, 136],
        [ 98, 136, 174]],

       [[ 63, 103, 143],
        [103, 143, 183]],

       [[ 66, 108, 150],
        [108, 150, 192]]])

In [79]:
dB

array([38, 40, 42])

![Imgur](https://i.imgur.com/gtU9vzZl.jpg)!

In [20]:
def test():
    # 初期化
    dX = np.zeros_like(X)

    # 誤差の計算
    for i in range(dX.shape[0]):
        for j in range(dX.shape[1]):
            for k in range(dA.shape[0]):
                for s in range(w.shape[2]):
                    # 入力の枠外の微分は0とする
                    if (j-s < 0) or (j-s > dA.shape[1]-1):
                        pass
                    # 枠内の微分値は足し合わせる
                    else:
                        dX[i, j] += dA[k, j-s] * w[k, i, s]
                        
    return dX

In [21]:
test()

array([[ 51, 120, 120,  69],
       [ 51, 120, 120,  69]])

In [22]:
class Conv1d:
    
    def __init__(self, b_size, initializer, optimizer, n_in_channels=1, n_out_channels=1, pa=0):
        self.b_size = b_size
        self.optimizer = optimizer
        self.pa = pa
        self.W = initializer.W(n_out_channels, n_in_channels, b_size)
        self.B = initializer.B(n_out_channels)
        self.n_in_channels = n_in_channels
        self.n_out_channels = n_out_channels
        self.n_out = None
        
    def forward(self, X):
        
        #Forward Propagation
        self.n_in = X.shape[-1]
        self.n_out = output_size_calculation(self.n_in, self.b_size, self.pa)
        X = X.reshape(self.n_in_channels, self.n_in)
        self.X = np.pad(X, ((0,0), ((self.b_size-1), 0)))
        self.X1 = np.zeros((self.n_in_channels, self.b_size, self.n_in+(self.b_size-1)))
        for i in range(self.b_size):
            self.X1[:, i] = np.roll(self.X, -i, axis=-1)
        A = np.sum(self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa]*self.W[:, :, :, np.newaxis], axis=(1, 2)) + self.B.reshape(-1,1)
        return A
    
    def backward(self, dA):
        
        #Back Propagation
        self.dW = np.sum(np.dot(dA, self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa, np.newaxis]), axis=-1)
        self.dB = np.sum(dA, axis=1)
        self.dA = np.pad(dA, ((0,0), (0, (self.b_size-1))))
        self.dA1 = np.zeros((self.n_out_channels, self.b_size, self.dA.shape[-1]))
        for i in range(self.b_size):
            self.dA1[:, i] = np.roll(self.dA, i, axis=-1)
        dX = np.sum(self.W@self.dA1, axis=0)
        self.optimizer.update(self)
        return dX

In [23]:
def output_size_calculation(n_in, F, P=0, S=1):
    n_out = int((n_in + 2*P - F) / S + 1)
    return n_out

In [24]:
test = Conv1d(b_size=3, n_in_channels=2, n_out_channels=3)

TypeError: __init__() missing 2 required positional arguments: 'initializer' and 'optimizer'

In [ ]:
testing = test.forward(np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) )
testing

In [ ]:
test.backward(dA=np.array([[16, 22], [17, 23], [18, 24]]))

In [ ]:
test.dW

In [ ]:
test.dB